## Next word prediction

Quick example code to obtain next word predictions of a sample of text.

First load and install the needed libraries

In [20]:
!pip install together

import string
from together import Together
import math


[notice] A new release of pip is available: 23.2.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


Copy key here. This key allows you to access a cloud computing service via an application programming interface (API).

In [31]:
key="X"

Write some text. This text will be used for the cloze task.

In [26]:
prompt = "The mouse, which eats the cheese."

Preprocess the text for the cloze.

In [27]:
text_list = prompt.split()

Define function for next token prediction

In [28]:
def next_token_probs(prompt: str, key: str):
    client = Together(api_key=key)                              # Define the client

    response = client.completions.create(                       # Define the prompt
        model="meta-llama/Llama-3.3-70B-Instruct-Turbo-Free",   # Choose the model
        prompt=prompt,                                          # Send the text
        max_tokens=1,                                           # Only predict the next token
        logprobs=5,                                             # Get top-k most likely tokens
    )

    # Get the Logprobs (logarithmic probabilities) Information object for the five most likely next tokens
    logprobs_part = response.choices[0].logprobs
    top_logprobs_dict = logprobs_part.top_logprobs[0] 

    # Convert logprobs to probabilities
    results = [(token, round(math.exp(lp), 4)) for token, lp in top_logprobs_dict.items()]

    return results

Run a loop for each word of the sentence, such that a cloze task can be implemented.

Output: List of word/probability pairs 

In [32]:
i = 0
for i in range(0,len(text_list)-1): # Loop to run through the sentence word by word
       prompt_text = " ".join(text_list[:i+1])  # Add one word per loop run
       print(prompt_text)                       # Show the text that is used for prompting
       print(next_token_probs(prompt_text, key))# Send the prompt and print it

The
[('Question', 0.3003), ('def', 0.1038), ('The', 0.046), ('#', 0.0359), ('This', 0.0162)]
The mouse,
[(' ', 0.0049), (' the', 0.0046), (' The', 0.0027), (' University', 0.0019), (' New', 0.0013)]
The mouse, which
[(' is', 0.1291), (' was', 0.0538), (' can', 0.037), (' has', 0.0348), (' may', 0.0152)]
The mouse, which eats
[(' the', 0.1312), (' a', 0.0483), (' insects', 0.0453), (' seeds', 0.0376), (' only', 0.0312)]
The mouse, which eats the
[(' seeds', 0.1232), (' seed', 0.04), (' eggs', 0.0312), (' insects', 0.0201), (' grass', 0.0189)]
